In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from SPModel import ReferEncoder
from SPModel import QueryEncoder
from SPModel import SequenceConvTransformer
from dataload import SPDataset

Using cpu device


In [2]:
trainnpdir = '../input/train_files/processed/'
train_sp_dir = trainnpdir+'train_sp_nonan.npy'

ReferLen = 1200
QueryLen = 60
QueryDelay = 20
Datalength = ReferLen + QueryDelay

train_ratio = 0.8

SPdataset = SPDataset(train_sp_dir)
datasettotallen = SPdataset.__len__()
trainlen = int(train_ratio*datasettotallen)
testlen = datasettotallen - trainlen
trainSPdataset, testSPdataset = torch.utils.data.random_split(SPdataset, [trainlen, testlen])

2496632 (2000, 10, 1258)
0 (76000, 8, 1220)


In [3]:
testSPdataloader = DataLoader(testSPdataset, batch_size=1, shuffle=True)

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

spmodel = SequenceConvTransformer().to(device)

Using cpu device


In [6]:
ckptpath = './models/ckpts/bestmodel_epoch_26_loss_0.000507.pt'
ckpt = torch.load(ckptpath, map_location=torch.device('cpu'))
spmodel.load_state_dict(ckpt['model_state_dict'])

<All keys matched successfully>

In [9]:
testnum = 100
correctcount = 0
meanerrorrate = 0
for i, (refer, query, target) in enumerate(testSPdataloader):
    refer, query, target = refer.to(device), query.to(device), target.to(device)
    spmodel.eval()
    pred = spmodel(refer, query).item()
    target = target.item()
    if pred * target >= 0:
        correctcount += 1
    if target != 0:
        meanerrorrate += abs(target-pred) / abs(target)

    print(i, 'Prediction:', pred, "Target:", target, end='\r')

    if i >= testnum-1:
        meanerrorrate /= testnum
        print()
        break
print(correctcount, testnum, correctcount/testnum)
print(meanerrorrate)

99 Prediction: -0.013305317610502243 Target: 0.03908794745802879316
67 100 0.67
1.716186054612532
